### Jupyter Notebook to understand TextAttack for Attacking Models 

We Plan to use 4 different Attack Strategies 

1) TextFoolerJin2019

2) DeepWordBugGao2018

3) BAEGarg2019

4) FasterGeneticAlgorithmJia2019

In [1]:
%load_ext autoreload
%autoreload 2


import logging
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

In [98]:
from ganbert_adversarial_attackutils import *

from textattack.attack_recipes import TextFoolerJin2019,DeepWordBugGao2018,BAEGarg2019,FasterGeneticAlgorithmJia2019, MorpheusTan2020, Pruthi2019

In [3]:
model_wrapper = BertModelWrapper(model_path='../gan-bert/gan_bert_fine_tuned_6652.pt')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


type of transformer:  <class 'transformers.models.bert.modeling_bert.BertModel'>
hiddensize is:  768
hidden_sizes[i] and [i+1] is 768 and 768
type of discriminator:  <class 'ganbert_models.Discriminator'>
There are 1 GPU(s) available.
We will use the GPU: Tesla K80





In [105]:
sst2_dataset = HuggingFaceDataset("gpt3mix/sst2", split="train")

Using custom data configuration default
Reusing dataset sst2 (/home/ecbm4040/.cache/huggingface/datasets/sst2/default/0.0.0/90167692658fa4abca2ffa3ede1a43a71e2bf671078c5c275c64c4231d5a62fa)
textattack: Loading datasets dataset gpt3mix/sst2, split train.


In [21]:
# Create the recipe
textfooler_recipe = TextFoolerJin2019.build(model_wrapper)

textattack: Unknown if model of class <class 'ganbert_adversarial_attackutils.InferenceGANBert'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [22]:
attacker = Attacker(textfooler_recipe, sst2_dataset, 
                    AttackArgs(num_examples = 10 ,
                                   shuffle = True , # Shuffle Dataset 
                                   log_to_csv = "Ganbert_Attack.csv" , # Log Attack to CSV 
                                   disable_stdout = True,# Supress individual Attack Results 
                              ))




In [23]:
results = attacker.attack_dataset()

textattack: Logging to CSV at path Ganbert_Attack.csv
[Succeeded / Failed / Skipped / Total] 0 / 0 / 2 / 2:  20%|▏| 2/10 [00:00<00:00,

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

Exception ignored in: <function CapturableResourceDeleter.__del__ at 0x7fcbbc0721f0>
Traceback (most recent call last):
  File "/home/ecbm4040/climate_change/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py", line 202, in __del__
    self._destroy_resource()
  File "/home/ecbm4040/climate_change/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "/home/ecbm4040/climate_change/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 823, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/ecbm4040/climate_change/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 696, in _initialize
    self._stateful_fn._get_concrete_function_internal_garbage_collected(  # pylint: disable=protected-access
  File "/home/ecbm4040/climate_change/lib/python3.8/site-packages/tensorflow/python/eager/function.py", line 


+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 0     |
| Number of failed attacks:     | 2     |
| Number of skipped attacks:    | 8     |
| Original accuracy:            | 20.0% |
| Accuracy under attack:        | 20.0% |
| Attack success rate:          | 0.0%  |
| Average perturbed word %:     | nan%  |
| Average num. words per input: | 19.4  |
| Avg num queries:              | 191.0 |
+-------------------------------+-------+


### DeepWordBugGao2018

In [24]:
# Create the recipe
textfooler_recipe = DeepWordBugGao2018.build(model_wrapper)

textattack: Unknown if model of class <class 'ganbert_adversarial_attackutils.InferenceGANBert'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [25]:
attacker = Attacker(textfooler_recipe, sst2_dataset, 
                    AttackArgs(num_examples = 10 ,
                                   shuffle = True , # Shuffle Dataset 
                                   log_to_csv = "Ganbert_Attack.csv" , # Log Attack to CSV 
                                   disable_stdout = True,# Supress individual Attack Results 
                              ))




In [26]:
results = attacker.attack_dataset()

textattack: Logging to CSV at path Ganbert_Attack.csv
[Succeeded / Failed / Skipped / Total] 0 / 0 / 2 / 2:  20%|▏| 2/10 [00:00<00:00,

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 2 / 8 / 10: 100%|█| 10/10 [00:01<00:0



+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 0     |
| Number of failed attacks:     | 2     |
| Number of skipped attacks:    | 8     |
| Original accuracy:            | 20.0% |
| Accuracy under attack:        | 20.0% |
| Attack success rate:          | 0.0%  |
| Average perturbed word %:     | nan%  |
| Average num. words per input: | 19.4  |
| Avg num queries:              | 57.5  |
+-------------------------------+-------+


### BAEGarg2019

In [27]:
# Create the recipe
textfooler_recipe = BAEGarg2019.build(model_wrapper)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'ganbert_adversarial_attackutils.InferenceGANBert'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [28]:
attacker = Attacker(textfooler_recipe, sst2_dataset, 
                    AttackArgs(num_examples = 10 ,
                                   shuffle = True , # Shuffle Dataset 
                                   log_to_csv = "Ganbert_Attack.csv" , # Log Attack to CSV 
                                   disable_stdout = True,# Supress individual Attack Results 
                              ))




In [29]:
results = attacker.attack_dataset()

textattack: Logging to CSV at path Ganbert_Attack.csv
[Succeeded / Failed / Skipped / Total] 0 / 0 / 2 / 2:  20%|▏| 2/10 [00:00<00:00,

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



Exception ignored in: <function CapturableResourceDeleter.__del__ at 0x7fcbbc0721f0>
Traceback (most recent call last):
  File "/home/ecbm4040/climate_change/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py", line 202, in __del__
    self._destroy_resource()
  File "/home/ecbm4040/climate_change/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "/home/ecbm4040/climate_change/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 823, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/ecbm4040/climate_change/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 696, in _initialize
    self._stateful_fn._get_concrete_function_internal_garbage_collected(  # pylint: disable=protected-access
  File "/home/ecbm4040/climate_change/lib/python3.8/site-packages/tensorflow/python/eager/function.py", line 


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 8      |
| Original accuracy:            | 20.0%  |
| Accuracy under attack:        | 10.0%  |
| Attack success rate:          | 50.0%  |
| Average perturbed word %:     | 45.45% |
| Average num. words per input: | 19.4   |
| Avg num queries:              | 198.0  |
+-------------------------------+--------+


In [70]:
results[3].goal_function_result_str()

'Positive (86%) --> [SKIPPED]'

In [72]:
results[3].num_queries

1

In [74]:
results[3].original_text()

"Francophiles will snicker knowingly and you 'll want to slap them ."

In [73]:
results[3].perturbed_text()

"Francophiles will snicker knowingly and you 'll want to slap them ."

In [50]:
results[0].perturbed_result.goal_status

3

In [51]:
results[0].perturbed_result.score

0.8515492081642151

In [53]:
results[0].original_result.score

0.8515492081642151

In [55]:
results[0].diff_color()

('These spiders can outrun a motorcycle and wrap a person in a sticky cocoon in seconds , but they fall short of being interesting or entertaining .',
 'These spiders can outrun a motorcycle and wrap a person in a sticky cocoon in seconds , but they fall short of being interesting or entertaining .')

In [57]:
results[0].str_lines()

('Positive (84%) --> [SKIPPED]',
 'These spiders can outrun a motorcycle and wrap a person in a sticky cocoon in seconds , but they fall short of being interesting or entertaining .')

### FasterGeneticAlgorithmJia2019

In [33]:
# Create the recipe
textfooler_recipe = FasterGeneticAlgorithmJia2019.build(model_wrapper)

/home/ecbm4040/climate_change/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.01 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
textattack: Unknown if model of class <class 'ganbert_adversarial_attackutils.InferenceGANBert'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [34]:
attacker = Attacker(textfooler_recipe, sst2_dataset, 
                    AttackArgs(num_examples = 10 ,
                                   shuffle = True , # Shuffle Dataset 
                                   log_to_csv = "Ganbert_Attack.csv" , # Log Attack to CSV 
                                   disable_stdout = True,# Supress individual Attack Results 
                              ))




In [75]:
results = attacker.attack_dataset()

[Succeeded / Failed / Skipped / Total] 0 / 0 / 2 / 2:  20%|▏| 2/10 [00:00<00:00,

Attack(
  (search_method): AlzantotGeneticAlgorithm(
    (pop_size):  60
    (max_iters):  20
    (temp):  0.3
    (give_up_if_no_improvement):  False
    (post_crossover_check):  False
    (max_crossover_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  8
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.2
        (compare_against_original):  True
      )
    (1): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (max_mse_dist):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (2): LearningToWriteLanguageModel(
        (max_log_prob_diff):  5.0
        (compare_against_original):  True
      )
    (3): RepeatModification
    (4): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 2 / 8 / 10: 100%|█| 10/10 [08:02<00:0
/home/ecbm4040/climate_change/lib/python3.8/site-packages/textattack/metrics/attack_metrics/words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
/home/ecbm4040/climate_change/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 0       |
| Number of failed attacks:     | 2       |
| Number of skipped attacks:    | 10      |
| Original accuracy:            | 16.67%  |
| Accuracy under attack:        | 16.67%  |
| Attack success rate:          | 0.0%    |
| Average perturbed word %:     | nan%    |
| Average num. words per input: | 19.75   |
| Avg num queries:              | 10507.5 |
+-------------------------------+---------+


In [81]:
results[2].str_lines()

('Positive (84%) --> [SKIPPED]',
 'These spiders can outrun a motorcycle and wrap a person in a sticky cocoon in seconds , but they fall short of being interesting or entertaining .')

### Morpheus

In [101]:
textattack_recipe = MorpheusTan2020.build(model_wrapper)

attacker = Attacker(textattack_recipe, sst2_dataset, 
                    AttackArgs(num_examples = 10 ,
                                   shuffle = True , # Shuffle Dataset 
                                   log_to_csv = "Ganbert_Attack.csv" , # Log Attack to CSV 
                                   disable_stdout = True,# Supress individual Attack Results 
                              ))

textattack: Unknown if model of class <class 'ganbert_adversarial_attackutils.InferenceGANBert'> compatible with goal function <class 'textattack.goal_functions.text.minimize_bleu.MinimizeBleu'>.


In [102]:
results = attacker.attack_dataset()

textattack: Logging to CSV at path Ganbert_Attack.csv

  0%|                                                    | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  MinimizeBleu(
    (maximizable):  False
    (target_bleu):  0.0
  )
  (transformation):  WordSwapInflections
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



TypeError: Invalid text_input type <class 'torch.Tensor'> (required str or OrderedDict)

In [91]:
results[4].goal_function_result_str()

'Positive (69%) --> [FAILED]'

In [93]:
results[4].perturbed_text()

'Engagingly captures the maddening and magnetic ebb and flows of friendship .'

In [94]:
results[4].original_text()

'Engagingly captures the maddening and magnetic ebb and flow of friendship .'

### HotFlip

In [106]:
textattack_recipe = Pruthi2019.build(model_wrapper)

attacker = Attacker(textattack_recipe, sst2_dataset, 
                    AttackArgs(num_examples = 10 ,
                                   shuffle = True , # Shuffle Dataset 
                                   log_to_csv = "Ganbert_Attack.csv" , # Log Attack to CSV 
                                   disable_stdout = True,# Supress individual Attack Results 
                              ))

textattack: Unknown if model of class <class 'ganbert_adversarial_attackutils.InferenceGANBert'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [107]:
results = attacker.attack_dataset()

textattack: Logging to CSV at path Ganbert_Attack.csv


  0%|                                                    | 0/10 [00:00<?, ?it/s]

[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%| | 1/10 [00:00<00:00,

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  False
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  False
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  False
      )
    (3): WordSwapQWERTY
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  1
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
) 





[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  20%|▏| 2/10 [00:01<00:07,

[Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:  20%|▏| 2/10 [00:01<00:07,

[Succeeded / Failed / Skipped / Total] 0 / 2 / 1 / 3:  30%|▎| 3/10 [00:04<00:09,

[Succeeded / Failed / Skipped / Total] 0 / 2 / 1 / 3:  40%|▍| 4/10 [00:04<00:06,

[Succeeded / Failed / Skipped / Total] 0 / 2 / 2 / 4:  40%|▍| 4/10 [00:04<00:06,

[Succeeded / Failed / Skipped / Total] 1 / 2 / 2 / 5:  50%|▌| 5/10 [00:08<00:08,

[Succeeded / Failed / Skipped / Total] 1 / 2 / 2 / 5:  60%|▌| 6/10 [00:10<00:06,

[Succeeded / Failed / Skipped / Total] 1 / 3 / 2 / 6:  60%|▌| 6/10 [00:10<00:06,

[Succeeded / Failed / Skipped / Total] 1 / 3 / 3 / 7:  70%|▋| 7/10 [00:10<00:04,

[Succeeded / Failed / Skipped / Total] 1 / 3 / 3 / 7:  80%|▊| 8/10 [00:11<00:02,

[Succeeded / Failed / Skipped / Total] 1 / 4 / 3 / 8:  80%|▊| 8/10 [00:11<00:02,

  0%|                                                    | 0/10 [02:46<?, ?it/s]


[Succeeded / 


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 5      |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 60.0%  |
| Accuracy under attack:        | 50.0%  |
| Attack success rate:          | 16.67% |
| Average perturbed word %:     | 4.0%   |
| Average num. words per input: | 14.2   |
| Avg num queries:              | 263.67 |
+-------------------------------+--------+


In [109]:
results[1].original_text()

'This version does justice both to Stevenson and to the sci-fi genre .'